In [3]:
from pathlib import Path
from matplotlib.font_manager import fontManager
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import seaborn as sns
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
rng=np.random.default_rng(seed=42)
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from utils.helper import fn_plot_torch_hist,fn_plot_confusion_matrix

from sklearn.preprocessing import StandardScaler


ModuleNotFoundError: No module named 'tensorflow'

In [8]:
###----------------------
### Some basic parameters
###----------------------

inpDir = Path('..') / '..' / 'input'
outDir = Path('..') / 'output'
modelDir = Path('..') / 'models'
subDir = 'fifa_2019'

RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE)
rng = np.random.default_rng(seed = RANDOM_STATE) # Set Random Seed for reproducible  results

EPOCHS = 100 # number of epochs
BATCH_SIZE = 32
ALPHA = 0.001 # learning rate
TEST_SIZE = 0.2
TRAIN_SIZE=454*BATCH_SIZE

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 6),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

plt.rcParams.update(params)

CMAP = plt.cm.coolwarm
plt.style.use('seaborn-v0_8-darkgrid') # plt.style.use('ggplot')

In [9]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# Check if all directories are present
outDir.mkdir(parents=True, exist_ok=True)

modelSubDir = modelDir/ subDir
modelSubDir.mkdir(parents=True, exist_ok=True)

In [13]:
data_df = pd.read_csv(r"D:\dnn_input\fifa_2019.csv")
data_df.shape

(18207, 89)

In [14]:
# removing rows with position = null
data_df = data_df[data_df["Position"].notnull()]
data_df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [15]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18147 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18147 non-null  int64  
 1   ID                        18147 non-null  int64  
 2   Name                      18147 non-null  object 
 3   Age                       18147 non-null  int64  
 4   Photo                     18147 non-null  object 
 5   Nationality               18147 non-null  object 
 6   Flag                      18147 non-null  object 
 7   Overall                   18147 non-null  int64  
 8   Potential                 18147 non-null  int64  
 9   Club                      17918 non-null  object 
 10  Club Logo                 18147 non-null  object 
 11  Value                     18147 non-null  object 
 12  Wage                      18147 non-null  object 
 13  Special                   18147 non-null  int64  
 14  Preferred F

In [16]:
data_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,18147.0,9089.239599,5257.923360,0.0,4536.5,9076.0,13662.5,18206.0
ID,18147.0,214279.818868,29970.319078,16.0,200305.5,221740.0,236512.5,246620.0
Age,18147.0,25.121122,4.669796,16.0,21.0,25.0,28.0,45.0
Overall,18147.0,66.253926,6.913320,46.0,62.0,66.0,71.0,94.0
Potential,18147.0,71.324076,6.132286,48.0,67.0,71.0,75.0,95.0
Special,18147.0,1598.002645,272.882706,731.0,1457.0,1635.0,1787.0,2346.0
International Reputation,18147.0,1.113297,0.394150,1.0,1.0,1.0,1.0,5.0
Weak Foot,18147.0,2.947154,0.660498,1.0,3.0,3.0,3.0,5.0
Skill Moves,18147.0,2.361492,0.756274,1.0,2.0,2.0,3.0,5.0
Jersey Number,18147.0,19.546096,15.947765,1.0,8.0,17.0,26.0,99.0


In [17]:
data_df.isna().sum()[data_df.isna().sum() > 0]

Club                      229
Joined                   1493
Loaned From             16883
Contract Valid Until      229
LS                       2025
ST                       2025
RS                       2025
LW                       2025
LF                       2025
CF                       2025
RF                       2025
RW                       2025
LAM                      2025
CAM                      2025
RAM                      2025
LM                       2025
LCM                      2025
CM                       2025
RCM                      2025
RM                       2025
LWB                      2025
LDM                      2025
CDM                      2025
RDM                      2025
RWB                      2025
LB                       2025
LCB                      2025
CB                       2025
RCB                      2025
RB                       2025
Release Clause           1504
dtype: int64

In [18]:
# Following columns appear to be relevant for our analysis
rel_cols = ["Position", 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
            'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
            'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
            'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
            'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
            'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
            'GKKicking', 'GKPositioning', 'GKReflexes']

In [19]:
goalkeeper = 'GK'
forward = ['ST', 'LW', 'RW', 'LF', 'RF', 'RS','LS', 'CF']
midfielder = ['CM','RCM','LCM', 'CDM','RDM','LDM', 'CAM', 'LAM', 'RAM', 'RM', 'LM']
defender = ['CB', 'RCB', 'LCB', 'LWB', 'RWB', 'LB', 'RB']

In [20]:
data_df=data_df[rel_cols]
data_df=data_df[data_df['Position'].notnull()]
data_df.head()

,Position,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,RF,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,ST,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,LW,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,GK,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,RCM,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [21]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18147 entries, 0 to 18206
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Position         18147 non-null  object 
 1   Finishing        18147 non-null  float64
 2   HeadingAccuracy  18147 non-null  float64
 3   ShortPassing     18147 non-null  float64
 4   Volleys          18147 non-null  float64
 5   Dribbling        18147 non-null  float64
 6   Curve            18147 non-null  float64
 7   FKAccuracy       18147 non-null  float64
 8   LongPassing      18147 non-null  float64
 9   BallControl      18147 non-null  float64
 10  Acceleration     18147 non-null  float64
 11  SprintSpeed      18147 non-null  float64
 12  Agility          18147 non-null  float64
 13  Reactions        18147 non-null  float64
 14  Balance          18147 non-null  float64
 15  ShotPower        18147 non-null  float64
 16  Jumping          18147 non-null  float64
 17  Stamina          

In [22]:
data_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Finishing,18147.0,45.550229,19.527445,2.0,30.0,49.0,62.0,95.0
HeadingAccuracy,18147.0,52.300766,17.381753,4.0,44.0,56.0,64.0,94.0
ShortPassing,18147.0,58.695432,14.696075,7.0,54.0,62.0,68.0,93.0
Volleys,18147.0,42.912217,17.695900,4.0,30.0,44.0,57.0,90.0
Dribbling,18147.0,55.375158,18.912224,4.0,49.0,61.0,68.0,97.0
Curve,18147.0,47.176283,18.396009,6.0,34.0,48.0,62.0,94.0
FKAccuracy,18147.0,42.866038,17.480034,3.0,31.0,41.0,57.0,94.0
LongPassing,18147.0,52.721386,15.325211,9.0,43.0,56.0,64.0,93.0
BallControl,18147.0,58.374828,16.685643,5.0,54.0,63.0,69.0,96.0
Acceleration,18147.0,64.612829,14.930320,12.0,57.0,67.0,75.0,97.0


In [23]:
for col in data_df.columns:
    print(f'Col: {col} -{data_df[col].unique()}')

Col: Position -['RF' 'ST' 'LW' 'GK' 'RCM' 'LF' 'RS' 'RCB' 'LCM' 'CB' 'LDM' 'CAM' 'CDM'
 'LS' 'LCB' 'RM' 'LAM' 'LM' 'LB' 'RDM' 'RW' 'CM' 'RB' 'RAM' 'CF' 'RWB'
 'LWB']
Col: Finishing -[95. 94. 87. 13. 82. 84. 72. 93. 60. 11. 91. 76. 42. 65. 90. 14. 67. 89.
 33. 88. 59. 83. 77. 79. 80. 55. 70. 86. 10. 92. 38. 15. 51. 19. 73. 56.
 64. 81. 75. 52. 40. 69. 46. 48. 45. 47. 66. 78. 63. 54. 12. 34. 57. 43.
 37. 20. 41. 85. 18. 25.  9. 71. 44. 27. 21. 74. 31. 68. 39. 62. 29. 61.
 32. 50. 16. 53.  8. 23. 58. 30. 49. 17. 36. 28. 35. 26.  6. 22. 24.  7.
  5.  2.  4.]
Col: HeadingAccuracy -[70. 89. 62. 21. 55. 61. 77. 91. 15. 85. 54. 92. 68. 84. 11. 13. 25. 83.
 59. 76. 56. 52. 48. 79. 87. 75. 10. 80. 81. 73. 86. 35. 74. 58. 72. 51.
 14. 82. 38. 50. 49. 60. 63. 19. 16. 46. 94. 69. 65. 12. 78. 90. 31. 41.
 43. 64. 42. 53. 17. 44. 45. 34. 67. 23. 57. 88. 71. 47. 40. 66. 36. 20.
 39. 93.  8. 18.  7. 37. 32. 24. 33.  9. 30. 22. 28. 27. 29. 26.  5.  6.
  4.]
Col: ShortPassing -[90. 81. 84. 50. 92. 89. 93

In [24]:
data_df.isnull().sum()

Position           0
Finishing          0
HeadingAccuracy    0
ShortPassing       0
Volleys            0
Dribbling          0
Curve              0
FKAccuracy         0
LongPassing        0
BallControl        0
Acceleration       0
SprintSpeed        0
Agility            0
Reactions          0
Balance            0
ShotPower          0
Jumping            0
Stamina            0
Strength           0
LongShots          0
Aggression         0
Interceptions      0
Positioning        0
Vision             0
Penalties          0
Composure          0
Marking            0
StandingTackle     0
SlidingTackle      0
GKDiving           0
GKHandling         0
GKKicking          0
GKPositioning      0
GKReflexes         0
dtype: int64

In [31]:
#Assign labels to goalkeepers
data_df.loc[data_df["Position"] == "GK", "Position"] = 0
#Defenders
data_df.loc[data_df["Position"].isin(defender), "Position"] = 1
#Midfielders
data_df.loc[data_df["Position"].isin(midfielder), "Position"] = 2
#Forward
data_df.loc[data_df["Position"].isin(forward), "Position"] = 3

In [32]:
# Convert Column "Position" to numeric so that Pandas does not complain
data_df['Position'] = pd.to_numeric(data_df['Position'], downcast="integer")

In [33]:
class_labels = {0: 'Goal Keeper', 1: 'Defender', 2: 'Mid-Fielder', 3: 'Forward'}

labels=data_df['Position']
features=data_df.drop('Position',axis=1)
labels.shape,features.shape

((18147,), (18147, 33))

In [34]:
train_df,test_df=train_test_split(data_df,stratify=data_df["Position"],train_size=TRAIN_SIZE,random_state=RANDOM_STATE)
train_df.shape,test_df.shape

((14528, 34), (3619, 34))

# GlobalScaler

In [36]:
class GlobalScaler:
    _instance=None

    def __init__(self):
        if GlobalScaler._instance is not None:
            raise Exception("GlobalScaler is Singleton Class")
        self.scaler=StandardScaler()

    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            cls._instance=GlobalScaler()
        return cls._instance


In [43]:
class FifaDS(Dataset):
    globalScaler=GlobalScaler.get_instance() #Protected Variable

    def __init__(self,dataframe,device=device,is_train=True,label_col=None):
        self.df=dataframe
        self.device=device
        self.is_train=is_train
        self.scaler=self.globalScaler.scaler
        self.label_col=label_col
    
        self.labels=self.df[label_col].to_numpy()
        if self.is_train:
            X=self.df.drop(label_col,axis=1)
            self.features=self.scaler.fit_transform(X)
        else:
            X=self.df.drop(label_col,axis=1)
            self.features=self.scaler.transform(X)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        feature=self.features[index]
        label=self.labels[index]
        # return super().__getitem__(index)
        return feature,label

In [ ]:

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
ttrain_loader = DataLoader(train_ds,batch_size = BATCH_SIZE,shuffle = True)
test_loader = DataLoader(test_ds,batch_size = BATCH_SIZE,shuffle = True)
for count,data in enumerate(train_loader):
    feat, lbl = dataprint(f'Count: {count}| {feat.shape},{lbl.shape}')

In [ ]:
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [ ]:
type(X_test),type(X_train),type(y_train),type(y_test)

In [ ]:
next(iter(train_ds))

In [ ]:
for features,labels in train_ds.take(5):
    print(features.shape,labels.shape)

# Define Modelm

In [ ]:
class Model(nn.Module):
    def __init__(self,input_dim):
        super().__init__()
                           #output 04
        out_dim_1 = 66
        out_dim_2 = 33
        out_dim_3 = 16
        out_dim_4 = 8
        out_dim_5 = 4

        self.laye1 = nn.Linear(input_dim,out_dim_1)
        self.actv1 = nn.ReLU()                          #output 33

        self.laye2 = nn.Linear(out_dim_1,out_dim_2)
        self.actv2 = nn.ReLU()                          #output 66

        self.laye3 = nn.Linear(out_dim_2,out_dim_3)
        self.actv3 = nn.ReLU()                          #output 33

        self.laye4 = nn.linear(out_dim_3,out_dim_4)
        self.actv4 = nn.ReLU()                          #output 16

        self.laye5 = nn.Linear(out_dim_4,out_dim_5)
        self.actv5 = nn.ReLU()                          #output 08

        self.layer6 = nn.Linear(out_dim_5,out_dim_6)
        self.actv6 = nn.ReLU()
    def forward(self,x):
        x = self.actv1(self.layer1(x))
        x = self.actv2(self.layer2(x))
        x = self.actv3(self.layer3(x))
        x = self.actv4(self.layer4(x))
        x = self.layer5(x)
model = Model(input_dim = 33).to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = ALPHA)

loss,tloss = [],[]
acc,tacc = [],[]
n_epoch = []

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    train_acc = 0.0

    for i,data in enumerate(train_loader):
        inputs,labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        preds = torch.argmax(outputs,dim = 1)

        batch_loss = loss_fn(outputs,labels)
        batch_oss.backward()
        optimizer.step()


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = ALPHA)

loss,tloss = [],[]
acc,tacc = [],[]
n_epoch = []

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    train_acc = 0.0

    for i,data in enumerate(train_loader):
        inputs,labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        preds = torch.argmax(outputs,dim = 1)

        optimizer.step()
        train_loss += batch_loss.item()*inputs.size(0)
        train_acc += batch_acc * inputs.size(0)

    train_loss = train_loss/len(train_ds)
    tloss.append(train_loss)
    train_acc = train_acc/len(train_loss)
    acc.append(train_acc)
n_epoch.append(epoch)
if epoch%10 ==0:
    print(f'Epoch: {epoch} | Loss: {train_loss:.4f} - {test_loss:.4f}')

In [ ]:
y_true,y_pred=[],[]
for feat,lbls in train_ds:
    pred=model(feat).numpy()
    y_true.extend(lbls)
    y_pred.extend(pred.argmax(axis=1))
print(f'Accuracy: {accuracy_score(y_true,y_pred):.4f}')
print(classification_report(y_true,y_pred))
##%%
fn_plot_confusion_matrix(y_true,y_pred, data_df['Position'].unique())
##%%
y_true,y_pred=[],[]
for feat,lbls in test_ds:
    pred=model(feat).numpy()
    y_true.extend(lbls)
    y_pred.extend(pred.argmax(axis=1))
print(f'Accuracy: {accuracy_score(y_true,y_pred):.4f}')
print(classification_report(y_true,y_pred))
##%%
fn_plot_confusion_matrix(y_true,y_pred, data_df['Position'].unique())